In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

import lsst.daf.butler as dB
import lsst.cp.verify.notebooks.utils as utils
import lsst.afw.display as afwDisplay
plt.rcParams['figure.figsize'] = (8, 8)

In [ ]:
# This cell contains parameters that can be automatically set via the papermill package.
# Examples:
#   papermill --prepare-only -p calibType newBias -p cameraName LSSTCam <input> <output>
#   papermill -p interactive False <input> <output>
interactive = True

# Which repository to use.
repository = '/repo/main/'

# Which calibration type to analyse.
calibType = 'bfk'
detectorNumber = 0

# Which camera the calibration is for.
cameraName = 'LATISS'

# Which display to use.
displayBackend = 'astrowidgets'

# Which collection the calibration was constructed in.
genCollection = 'u/czw/DM-28920/bfkGen.20210928-exp'

# Which collection the PTC data was constructed in.
ptcCollection = 'u/czw/DM-28920/ptcGen.20210723Ya'

# Which collection containing the verification outputs.
correctedCollection = 'u/czw/DM-30172/bfkV.20211102Xa'

In [ ]:
# Get butler and camera
butler = dB.Butler(repository, collections=[genCollection, ptcCollection,
                                            correctedCollection])
camera = butler.get('camera', instrument=cameraName)

In [ ]:
# Get the calibrations.
bfk = butler.get(calibType, instrument=cameraName, detector=detectorNumber,
                 collection=genCollection)
ptc = butler.get('ptc', instrument=cameraName, detector=detectorNumber,
                 collection=ptcCollection)

In [ ]:
runStats = butler.get('verifyBfkStats', instrument=cameraName, 
                      collections=correctedCollection)
runSuccess = runStats.pop('SUCCESS')

In [ ]:
# Display summary table of tests and failure counts.
utils.failureTable(runStats)

In [ ]:
# Plot raw cross-correlation values
inputSelection = 11
iterator = 0
fig, axes = plt.subplots(nrows=4, ncols=4,
                         sharex=True, sharey=False, figsize=[8.0, 8.0], 
                         constrained_layout=True)
for ampName in sorted(set(ptc.ampNames)):
    covariance = ptc.covariances[ampName][inputSelection]
    axes[iterator // 4, iterator % 4].set_aspect("equal")
    axes[iterator // 4, iterator % 4].imshow(np.log10(np.abs(covariance)))
    axes[iterator // 4, iterator % 4].set_title(f"Amp {ampName} {covariance[0][0]:.2e}")

    iterator += 1
fig.suptitle(f"Measured covariance {ptc.inputExpIdPairs[ampName][inputSelection]}")
plt.show()

In [ ]:
# Plot mean cross-correlation values
iterator = 0
fig, axes = plt.subplots(nrows=4, ncols=4,
                         sharex=True, sharey=False, figsize=[8.0, 8.0], 
                         constrained_layout=True)
for ampName in sorted(set(ptc.ampNames)):
    axes[iterator // 4, iterator % 4].set_aspect("equal")
    axes[iterator // 4, iterator % 4].imshow(np.log10(np.abs(bfk.meanXcorrs[ampName])))
    axes[iterator // 4, iterator % 4].set_title(f"Amp {ampName} {bfk.meanXcorrs[ampName][8, 8]:.2e}")

    iterator += 1
fig.suptitle("Mean cross-correlations")
plt.show()

In [ ]:
# Plot cuts across mean cross-correlations
iterator = 0
fig, axes = plt.subplots(nrows=4, ncols=4,
                         sharex=True, sharey=False, figsize=[8.0, 8.0], 
                         constrained_layout=True)
for ampName in sorted(set(ptc.ampNames)):
    axes[iterator // 4, iterator % 4].set_title(f"Amp {ampName} {bfk.meanXcorrs[ampName][8, 8]:.2e}")

    axes[iterator // 4, iterator % 4].step(range(0, 17), bfk.meanXcorrs[ampName][:, 8],
                                           where='mid', label='Y-axis')
    axes[iterator // 4, iterator % 4].step(range(0, 17), bfk.meanXcorrs[ampName][8, :],
                                           where='mid', label='X-axis')
    axes[iterator // 4, iterator % 4].set_xlabel("Location (pxl)")
    axes[iterator // 4, iterator % 4].set_ylabel("Mean cross-corr")
    axes[iterator // 4, iterator % 4].legend()
    iterator += 1
fig.suptitle("Central cuts of mean cross-correlation")
plt.show()

In [ ]:
# Plot average kernel <-- This is what gets used in IsrTask
detector = camera[0]
detName = detector.getName()
bfk.makeDetectorKernelFromAmpwiseKernels(detName)
averageKernel = bfk.detKernels[detName]
plt.imshow(averageKernel)
plt.title("Detector averaged kernel")
plt.show()

In [ ]:
# Plot central cuts of detector averaged kernel.
plt.step(range(0, 17), bfk.detKernels[detName][:, 8], 
         where='mid', label='Y-axis')
plt.step(range(0, 17), bfk.detKernels[detName][8, :], 
         where='mid', label='X-axis')
plt.legend()
plt.xlabel("Location (pxl)")
plt.ylabel("Kernel value")
plt.title("Central cuts of detector averaged kernel")
plt.show()

In [ ]:
# Plot amp kernels
iterator = 0
fig, axes = plt.subplots(nrows=4, ncols=4,
                         sharex=True, sharey=False, figsize=[8.0, 8.0], 
                         constrained_layout=True)
for ampName in sorted(set(ptc.ampNames)):
    axes[iterator // 4, iterator % 4].set_aspect("equal")
    axes[iterator // 4, iterator % 4].imshow(bfk.ampKernels[ampName])
    axes[iterator // 4, iterator % 4].set_title(f"Amp {ampName} {bfk.ampKernels[ampName][8, 8]:.2e}")

    iterator += 1
fig.suptitle("Amplfier level kernels")
plt.show()

In [ ]:
# Plot cuts across the amp kernels
iterator = 0
fig, axes = plt.subplots(nrows=4, ncols=4,
                         sharex=True, sharey=False, figsize=[8.0, 8.0], 
                         constrained_layout=True)
for ampName in sorted(set(ptc.ampNames)):
    axes[iterator // 4, iterator % 4].set_title(f"Amp {ampName} {bfk.ampKernels[ampName][8, 8]:.2e}")

    axes[iterator // 4, iterator % 4].step(range(0, 17), bfk.ampKernels[ampName][:, 8], 
                                           where='mid', label='Y-axis')
    axes[iterator // 4, iterator % 4].step(range(0, 17), bfk.ampKernels[ampName][8, :], 
                                           where='mid', label='X-axis')
    axes[iterator // 4, iterator % 4].set_xlabel("Location (pxl)")
    axes[iterator // 4, iterator % 4].set_ylabel("Mean cross-corr")
    axes[iterator // 4, iterator % 4].legend()

    iterator += 1
plt.show()

In [ ]:
# Get object information and sizes.
Fall = []
Tall = []
TuAll = []
for detector in camera:
    detId = detector.getId()
    for exposureId, stats in runStats.items():
        icSrc = butler.get('icSrc', collections=correctedCollection,
                           visit=exposureId, detector=detId)
        icSrcUn = butler.get('verifyUncorrBfkSrc', collections=correctedCollection, 
                             visit=exposureId, detector=detId)
        if len(icSrc) != len(icSrcUn):
            continue
        T = icSrc['base_SdssShape_xx'] + icSrc['base_SdssShape_yy']
        Tp = icSrc['base_SdssShape_psf_xx'] + icSrc['base_SdssShape_psf_yy']
        F = (icSrc['base_SdssShape_instFlux'])
        Tu = icSrcUn['base_SdssShape_xx'] + icSrcUn['base_SdssShape_yy']
        Tup = icSrcUn['base_SdssShape_psf_xx'] + icSrcUn['base_SdssShape_psf_yy']
        plt.xlabel("InstFlux")
        plt.ylabel("Uncorrected size - corrected size")
        plt.ylim(-10, 10)
        plt.scatter(F, 100*(Tu - T), label=exposureId, s=1.5)
        Fall.extend(F)
        Tall.extend(T)
        TuAll.extend(Tu)
    plt.legend()
    plt.xlim(0, 1e6)
    plt.title(detector.getName())
    plt.grid()
    plt.show()

In [ ]:
from scipy.optimize import least_squares
def f(x, a, b, c):
    return a * np.exp(b * (x + c))

def residual(p, x, y):
    return y - f(x, *p)

In [ ]:
# Plot functional fits.
XX = np.arange(-20, -11, 0.1)

for exposureId in runStats.keys():
    for detector in camera:
        detStats = butler.get("verifyBfkDetStats", instrument=cameraName, 
                              detector=detector.getId(),
                              exposure=exposureId, visit=exposureId)
        print(detStats['CATALOG']['BRIGHT_SLOPE'])
        M = detStats['CATALOG']['MAGNITUDES']
        D = detStats['CATALOG']['SIZE_DIFF']
        mask = np.isfinite(M) * np.isfinite(D)
        
        p0 = [0.0, 0.0, 0.0]
        ZZ = least_squares(residual, p0, args=(np.array(M)[mask], np.array(D)[mask]), 
                           loss='cauchy')
        pf = np.polyfit(np.array(M)[mask], np.array(D)[mask], 2)
        
        plt.scatter(detStats['CATALOG']['MAGNITUDES'], detStats['CATALOG']['SIZE_DIFF'])
        plt.plot(XX, np.polyval(pf, XX), label=f"polynomial (N=2) {pf}")
        plt.plot(XX, f(XX, *ZZ.x), label=f"exponential {ZZ.x}")
        plt.ylim(-1, 5)
        plt.legend()

        plt.title(exposureId)
        plt.show()